In [ ]:
# pull in the training set data
# remove any duplicates in the data
# split the data into 75% training, 25% test 
# parse the data into inputs and label arrays
# define the classification neural network (softmax is fine to start with)
# train the neural network (model.fit)
# test the neural network for accuracy on both the training & test data sets
# iterate on the neural network to achieve 80%+ accuracy 
#(if more data is needed then determine how much more data is needed)
# do another test with my most recent trading data and compare results

# then productionize it (should just be an api called check_trading_plan
# or grade_trading_plan) (remember that neural networks are easy to share
# after the weights & biases are generated)

In [8]:
import csv
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from enum import Enum


2023-12-22 21:53:24.141553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jabariholloway/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [35]:
class ClassLabel(Enum):
    HIGH_PROBABILITY = 2
    MED_PROBABILITY = 1
    LOW_PROBABILITY = 0

In [36]:
def unpack_training_data(file_name):
    training_data = []
    with open(file_name, newline='') as f:
        reader = csv.DictReader(f)

        for row in reader:
            training_data.append(row)

    # remove duplicates
    for i, datum in enumerate(training_data):
        j = i + 1
        if j < len(training_data) - 1:
            key1 = f"{datum['Symbol']}-{datum['Trade_Open_Datetime']}-{datum['Trade_Close_Datetime']}-{datum['Trade_PnL']}"

            datum2 = training_data[j]
            key2 = f"{datum2['Symbol']}-{datum2['Trade_Open_Datetime']}-{datum2['Trade_Close_Datetime']}-{datum2['Trade_PnL']}"

            if key1 == key2:
                datum2['duplicate'] = True

    training_data = [datum for datum in training_data if not datum.get('duplicate')]

    for datum in training_data:
        for key in datum.keys():
            if key.startswith("Is") or key.startswith("Label"):
                datum[key] = int(datum[key])

    return training_data

In [37]:
raw_training_data = unpack_training_data("training_data_2023-10-10_2023-11-17_complete.txt")
print(raw_training_data[0])
print(raw_training_data[len(raw_training_data) - 1])
# for datum in raw_training_data:
#     print(len(datum.values()))

{'Trade_Id': '1', 'Symbol': 'MESZ3', 'Trade_Open_Datetime': '2023-10-10T20:59:23', 'Trade_Close_Datetime': '2023-10-10T21:09:16', 'Trade_Quantity': '1', 'Trade_Direction': 'Long', 'Trade_Open_Price': '4393.50', 'Trade_Close_Price': '4393.50', 'Trade_PnL': '0.00', 'Is_Good_Mood': 1, 'Is_Good_Energy': 1, 'Is_Revenge_Trade': 0, 'Has_Trade_Plan': '0', 'Is_Automated_Trade_Mgmt': 0, 'Is_Alert_Based': 0, 'Is_Good_Trade_Setup': 0, 'Has_Confirmations': '0', 'Has_Good_Confirmations': '0', 'Has_Good_Risk_Reward_Ratio': '0', 'Is_Profitable_Trade': 2, 'Has_Followed_Plan_Completely': '0', 'Has_Followed_Plan_Partially': '0', 'Has_Exited_Early': '0', 'Has_Stopped_Out_Late': '0', 'Has_Not_Followed_Plan_At_All': '0', 'Is_Good_Time_Of_Day_To_Trade': 0, 'Is_Good_Market_Conditions': 0, 'Is_Good_Instrument_Conditions': 0, 'Is_Good_Financial_Instrument': 0, 'Is_Okay_Financial_Instrument': 1, 'Is_Bad_Financial_Instrument': 0, 'Is_Good_Position_Size': 1, 'Label_Is_High_Prob_Trade': 0, 'Label_Is_Med_Prob_Trade'

In [38]:
# Create inputs & labels
X = []
y = []

for datum in raw_training_data:
    X_row = []
    for key in datum.keys():
        if key.startswith("Is"):
            X_row.append(datum[key])

        if key.startswith("Label"):
            if key == 'Label_Is_High_Prob_Trade' and datum[key] == 1:
                y.append(ClassLabel.HIGH_PROBABILITY.value)
            elif key == 'Label_Is_Med_Prob_Trade' and datum[key] == 1:
                y.append(ClassLabel.MED_PROBABILITY.value)
            elif key == 'Label_Is_Low_Prob_Trade' and datum[key] == 1:
                y.append(ClassLabel.LOW_PROBABILITY.value)

    X.append(X_row)
            

print(len(X))
# columns = X[0]
print(len(X[0]))
print(len(y))
print(y)

136
14
136
[0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0, 1, 1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0]


In [39]:
# Split data into traing & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train[0])
print(y_train[0])

[1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1]
0


In [40]:
# Define Neural Network

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(14,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [41]:
# Compile Neural Network

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [42]:
# Train Model

model.fit(X_train, y_train, epochs=10)

Epoch 1/10
4/4 [==============================] - 0s 7ms/step - loss: 1.0000 - accuracy: 0.6078
Epoch 2/10
4/4 [==============================] - 0s 3ms/step - loss: 0.9303 - accuracy: 0.6373
Epoch 3/10
4/4 [==============================] - 0s 2ms/step - loss: 0.8780 - accuracy: 0.6667
Epoch 4/10
4/4 [==============================] - 0s 2ms/step - loss: 0.8408 - accuracy: 0.6667
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 0.8116 - accuracy: 0.6667
Epoch 6/10
4/4 [==============================] - 0s 3ms/step - loss: 0.7950 - accuracy: 0.6667
Epoch 7/10
4/4 [==============================] - 0s 2ms/step - loss: 0.7829 - accuracy: 0.6667
Epoch 8/10
4/4 [==============================] - 0s 3ms/step - loss: 0.7710 - accuracy: 0.6667
Epoch 9/10
4/4 [==============================] - 0s 2ms/step - loss: 0.7624 - accuracy: 0.6667
Epoch 10/10
4/4 [==============================] - 0s 2ms/step - loss: 0.7526 - accuracy: 0.6569


In [43]:
# Evaluate accuracy

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('Test accuracy:', test_acc)

2/2 - 0s - loss: 0.9578 - accuracy: 0.4706 - 150ms/epoch - 75ms/step
Test accuracy: 0.47058823704719543
